In [ ]:
# !pip install BeautifulSoup
!pip install beautifulsoup4

In [1]:
import os
import json
import time
import random
import zipfile
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Class Explanation: `NewsScraper`

## Overview
The `NewsScraper` class is designed for scraping news articles from three different Urdu news websites: Geo, Jang, and Express. The class has methods that cater to each site's unique structure and requirements. Below, we will go through the class and its methods, detailing what each function does, the input it takes, and the output it returns.

## Class Definition

```python
class NewsScraper:
    def __init__(self, id_=0):
        self.id = id_
```


## Method 1: `get_express_articles`

### Description
Scrapes news articles from the Express website across categories like saqafat (entertainment), business, sports, science-technology, and world. The method navigates through multiple pages for each category to gather a more extensive dataset.

### Input
- **`max_pages`**: The number of pages to scrape for each category (default is 7).

### Process
- Iterates over each category and page.
- Requests each category page and finds article cards within `<ul class='tedit-shortnews listing-page'>`.
- Extracts the article's headline, link, and content by navigating through `<div class='horiz-news3-caption'>` and `<span class='story-text'>`.

### Output
- **Returns**: A tuple of:
  - A Pandas DataFrame containing columns: `id`, `title`, and `link`).
  - A dictionary `express_contents` where the key is the article ID and the value is the article content.

### Data Structure
- Article cards are identified by `<li>` tags.
- Content is structured within `<span class='story-text'>` and `<p>` tags.



In [2]:
class NewsScraper:
    def __init__(self,id_=0):
        self.id = id_


  # write functions to scrape from other websites
    def get_geo_articles(self):
        geo_df = {
            "id": [],
            "title": [],
            "link": [],
            "content": [],
            "gold_label": [],
        }
        base_url = 'https://urdu.geo.tv/category/'
        categories = ['world', 'sports', 'entertainment', 'business', 'science-technology']
        num_pages = 2 

        for category in categories:
            print(f"Scraping articles from category '{category}'...")

            success_count = 0

            for page in range(1, num_pages + 1):
                print(f"  Scraping page {page} of '{category}'...")
                url = f"{base_url}{category}?page={page}"
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")

                cards = soup.find_all('div', class_='singleBlock')
                print(f"\t--> Found {len(cards)} articles on page {page} of '{category}'.")

                for card in cards:
                    try:
                        headline = card.find('h2').get_text(strip=True)

                        link = card.find('a', class_='open-section')['href']

                        article_response = requests.get(link)
                        article_response.raise_for_status()
                        content_soup = BeautifulSoup(article_response.text, "html.parser")

                        paras = content_soup.find_all('p')
                        combined_text = " ".join(
                            p.get_text(strip=True).replace('\xa0', ' ').replace('\u200b', '')
                            for p in paras if p.get_text(strip=True)
                        )

                        geo_df['id'].append(self.id)
                        geo_df['title'].append(headline)
                        geo_df['link'].append(link)
                        geo_df['gold_label'].append(category)
                        geo_df['content'].append(combined_text)

                        self.id += 1
                        success_count += 1

                    except Exception as e:
                        print(f"\t--> Failed to scrape an article on page {page} of '{category}': {e}")

            print(f"\t--> Successfully scraped {success_count} articles from '{category}'.\n")

        return pd.DataFrame(geo_df)

    def get_jang_articles(self):
        jang_df = {
            "id": [],
            "title": [],
            "link": [],
            "content": [],
            "gold_label": [],
        }
        base_url = 'https://jang.com.pk'
        categories = {
            'world': 'latest-news/world',
            'sports': 'latest-news/sports',
            'entertainment': 'latest-news/entertainment',
            'business': 'latest-news/business',
        }

        for category, path in categories.items():
            url = f"{base_url}/category/{path}"
            print(f"Scraping {category} articles from: {url}")
            
            try:
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")

                articles = soup.find_all('li')
                print(f"\t--> Found {len(articles)} articles in '{category}' category.")

                success_count = 0

                for article in articles:
                    try:
                        headline_tag = article.find('h2')
                        headline = headline_tag.get_text(strip=True) if headline_tag else None

                        link_tag = article.find('a', href=True)
                        link = link_tag['href'] if link_tag else None

                        if not headline or not link:
                            continue

                        if link and not link.startswith("http"):
                            link = f"{base_url}{link}"

                        article_response = requests.get(link)
                        article_response.raise_for_status()
                        article_soup = BeautifulSoup(article_response.text, "html.parser")

                        paras = article_soup.find_all('p')
                        combined_text = " ".join(
                            p.get_text(strip=True).replace('\xa0', ' ').replace('\u200b', '')
                            for p in paras if p.get_text(strip=True)
                        )

                        jang_df['id'].append(self.id)
                        jang_df['title'].append(headline)
                        jang_df['link'].append(link)
                        jang_df['gold_label'].append(category)
                        jang_df['content'].append(combined_text)

                        self.id += 1
                        success_count += 1

                    except Exception as e:
                        print(f"\t--> Failed to scrape an article in '{category}': {e}")

                print(f"\t--> Successfully scraped {success_count} articles from '{category}' category.\n")

            except Exception as e:
                print(f"Failed to retrieve category page for '{category}': {e}")
        
        return pd.DataFrame(jang_df)

    def get_dunya_articles(self, max_pages=7):
        dunya_df = {
            "id": [],
            "title": [],
            "link": [],
            "content": [],
            "gold_label": [],
        }

        base_url = 'https://urdu.dunyanews.tv'
        categories = ['Business', 'Sports', 'Entertainment', 'World', 'Technology']  # Add or update categories as needed

        for category in categories:
            print(f"Scraping articles from category '{category}'...")

            url = f"{base_url}/index.php/ur/{category}"
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            cards = soup.find_all('h3', class_='col-md-12 newsttl') + \
                    soup.find_all('div', class_='col-md-6 col-sm-6 col-xs-6') + \
                     soup.find_all('div', class_='col-md-8')


            print(f"\t--> Found {len(cards)} articles in '{category}'.")

            success_count = 0

            for card in cards:
                try:

                    if 'newsttl' in card.get('class', []):
                        title_tag = card.find('a')
                    else:
                        title_tag = card.find('h3').find('a')

                    title = title_tag.get_text(strip=True)
                    link_suffix = title_tag['href']
                    link = base_url + link_suffix if link_suffix.startswith('/') else link_suffix

                    article_response = requests.get(link)
                    article_response.raise_for_status()
                    content_soup = BeautifulSoup(article_response.text, "html.parser")

                    paras = content_soup.find('div', class_='main-news col-md-12').find_all('p')
                    combined_text = " ".join(
                        p.get_text(strip=True).replace('\xa0', ' ').replace('\u200b', '')
                        for p in paras if p.get_text(strip=True)
                    )

                    gold_label = category if category != 'Technology' else 'Science-Technology'

                    dunya_df['id'].append(self.id)
                    dunya_df['title'].append(title)
                    dunya_df['link'].append(link)
                    dunya_df['gold_label'].append(category)
                    dunya_df['content'].append(combined_text)

                    self.id += 1
                    success_count += 1

                except Exception as e:
                    print(f"\t--> Failed to scrape an article in '{category}': {e}")

            print(f"\t--> Successfully scraped {success_count} articles from '{category}'.\n")

        return pd.DataFrame(dunya_df)


    def get_express_articles(self, max_pages=7):
        express_df = {
            "id": [],
            "title": [],
            "link": [],
            "content": [],
            "gold_label": [],
        }
        base_url = 'https://www.express.pk'
        categories = ['saqafat', 'business', 'sports', 'science', 'world']   # saqafat is entertainment category

        # Iterating over the specified number of pages
        for category in categories:
            for page in range(1, max_pages + 1):
                print(f"Scraping page {page} of category '{category}'...")
                url = f"{base_url}/{category}/archives?page={page}"
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")

                # Finding article cards
                cards = soup.find('ul', class_='tedit-shortnews listing-page').find_all('li')  # Adjust class as per actual site structure
                print(f"\t--> Found {len(cards)} articles on page {page} of '{category}'.")

                success_count = 0

                for card in cards:
                    try:
                        div = card.find('div',class_='horiz-news3-caption')

                        # Article Title
                        headline = div.find('a').get_text(strip=True).replace('\xa0', ' ')

                        # Article link
                        link = div.find('a')['href']

                        # Requesting the content from each article's link
                        article_response = requests.get(link)
                        article_response.raise_for_status()
                        content_soup = BeautifulSoup(article_response.text, "html.parser")


                        # Content arranged in paras inside <span> tags
                        paras = content_soup.find('span',class_='story-text').find_all('p')

                        combined_text = " ".join(
                        p.get_text(strip=True).replace('\xa0', ' ').replace('\u200b', '')
                        for p in paras if p.get_text(strip=True)
                        )

                        # Storing data
                        express_df['id'].append(self.id)
                        express_df['title'].append(headline)
                        express_df['link'].append(link)
                        express_df['gold_label'].append(category.replace('saqafat','entertainment').replace('science','science-technology'))
                        express_df['content'].append(combined_text)

                        # Increment ID and success count
                        self.id += 1
                        success_count += 1

                    except Exception as e:
                        print(f"\t--> Failed to scrape an article on page {page} of '{category}': {e}")

                print(f"\t--> Successfully scraped {success_count} articles from page {page} of '{category}'.")
            print('')

        return pd.DataFrame(express_df)

In [3]:
scraper = NewsScraper()

In [26]:
express_df = scraper.get_express_articles()

Scraping page 1 of category 'saqafat'...
	--> Found 10 articles on page 1 of 'saqafat'.
	--> Successfully scraped 10 articles from page 1 of 'saqafat'.
Scraping page 2 of category 'saqafat'...
	--> Found 10 articles on page 2 of 'saqafat'.
	--> Successfully scraped 10 articles from page 2 of 'saqafat'.
Scraping page 3 of category 'saqafat'...
	--> Found 10 articles on page 3 of 'saqafat'.
	--> Successfully scraped 10 articles from page 3 of 'saqafat'.
Scraping page 4 of category 'saqafat'...
	--> Found 10 articles on page 4 of 'saqafat'.
	--> Successfully scraped 10 articles from page 4 of 'saqafat'.
Scraping page 5 of category 'saqafat'...
	--> Found 10 articles on page 5 of 'saqafat'.
	--> Successfully scraped 10 articles from page 5 of 'saqafat'.
Scraping page 6 of category 'saqafat'...
	--> Found 10 articles on page 6 of 'saqafat'.
	--> Successfully scraped 10 articles from page 6 of 'saqafat'.
Scraping page 7 of category 'saqafat'...
	--> Found 10 articles on page 7 of 'saqafat'.


In [4]:
geo_df = scraper.get_geo_articles()

Scraping articles from category 'world'...
  Scraping page 1 of 'world'...
	--> Found 60 articles on page 1 of 'world'.
  Scraping page 2 of 'world'...
	--> Found 60 articles on page 2 of 'world'.
	--> Successfully scraped 120 articles from 'world'.

Scraping articles from category 'sports'...
  Scraping page 1 of 'sports'...
	--> Found 60 articles on page 1 of 'sports'.
  Scraping page 2 of 'sports'...
	--> Found 60 articles on page 2 of 'sports'.
	--> Successfully scraped 120 articles from 'sports'.

Scraping articles from category 'entertainment'...
  Scraping page 1 of 'entertainment'...
	--> Found 60 articles on page 1 of 'entertainment'.
  Scraping page 2 of 'entertainment'...
	--> Found 60 articles on page 2 of 'entertainment'.
	--> Successfully scraped 120 articles from 'entertainment'.

Scraping articles from category 'business'...
  Scraping page 1 of 'business'...
	--> Found 60 articles on page 1 of 'business'.
  Scraping page 2 of 'business'...
	--> Found 60 articles on pag

In [5]:
geo_df.shape[0]

600

In [6]:
geo_df.head()

,id,title,link,content,gold_label
0,0,ٹرمپ کے متنازع احکامات پر کیا ردعمل دینا ہے؟ پ...,https://urdu.geo.tv/latest/386198-,09 نومبر ، 2024 امریکی وزارت دفاع پینٹاگون کے ...,world
1,1,انسانی روبوٹ کا پہلا فن پارہ 30کروڑ روپے میں ن...,https://urdu.geo.tv/latest/386187-,09 نومبر ، 2024 ایک انسان نما روبوٹ نے ایک انگ...,world
2,2,غزہ: مسلسل اسرائیلی بمباری سے گزشتہ 24 گھنٹوں ...,https://urdu.geo.tv/latest/386167-,09 نومبر ، 2024 غزہ میں مسلسل اسرائیلی بمباری ...,world
3,3,امریکا نے قطر سے حماس قیادت کو ملک بدر کرنے کا...,https://urdu.geo.tv/latest/386165-,09 نومبر ، 2024 امریکا نے اسرائیلی یرغمالیوں ک...,world
4,4,امریکی مسلمان اور عرب ووٹرز کے ڈیموکریٹک پارٹی...,https://urdu.geo.tv/latest/386163-,09 نومبر ، 2024 امریکا کی سب سے بڑی مسلم شہری ...,world


In [25]:
geo_df.tail()

,id,title,link,content,gold_label
595,595,گوگل میپس کے ڈیزائن میں تبدیلی کرنے کا فیصلہ,https://urdu.geo.tv/latest/383896-,20 اکتوبر ، 2024 اگر آپ گوگل میپس کو موسم کی ت...,science-technology
596,596,دنیا کا پہلا پرائیویٹ اسپیس اسٹیشن اندر سے کیس...,https://urdu.geo.tv/latest/383879-,20 اکتوبر ، 2024 متعدد افراد بچپن میں خلا میں ...,science-technology
597,597,ملازمین کی چھٹی، گھر کے کام کاج کرنیوالے روبوٹ...,https://urdu.geo.tv/latest/383799-,19 اکتوبر ، 2024 کیا آپ بھی آئے روز کام وال...,science-technology
598,598,ایکس (ٹوئٹر) کا نئی پرائیویسی پالیسی کے تحت صا...,https://urdu.geo.tv/latest/383713-,18 اکتوبر ، 2024 سوشل میڈیا پلیٹ فارم ایکس (ٹو...,science-technology
599,599,ایک سیکنڈ میں 20 سے زیادہ فلمیں ڈاؤن لوڈ کرنے ...,https://urdu.geo.tv/latest/383705-,18 اکتوبر ، 2024 ویسے تو 5 جی ٹیکنالوجی ہی دنی...,science-technology


In [18]:
jang_df = scraper.get_jang_articles()

Scraping world articles from: https://jang.com.pk/category/latest-news/world
	--> Found 237 articles in 'world' category.
	--> Successfully scraped 104 articles from 'world' category.

Scraping sports articles from: https://jang.com.pk/category/latest-news/sports
	--> Found 236 articles in 'sports' category.
	--> Successfully scraped 103 articles from 'sports' category.

Scraping entertainment articles from: https://jang.com.pk/category/latest-news/entertainment
	--> Found 236 articles in 'entertainment' category.
	--> Successfully scraped 103 articles from 'entertainment' category.

Scraping business articles from: https://jang.com.pk/category/latest-news/business
	--> Found 234 articles in 'business' category.
	--> Successfully scraped 101 articles from 'business' category.



In [8]:
jang_df.head()

,id,title,link,content,gold_label
0,600,ڈھاکا: بی این پی کا حسینہ واجد کے ٹرائل اور نئ...,https://jang.com.pk/news/1409381,بنگلا دیش کی بڑی سیاسی جماعت بنگلا دیش نیشنل پ...,world
1,601,اسرائیلی حملے، مزید 44 فلسطینی، 52 لبنانی شہید,https://jang.com.pk/news/1409379,غزہ کے مختلف علاقوں میں اسرائیلی جارحیت جاری ہ...,world
2,602,ٹرمپ کے مقابلے میں کملا ہیرس کو زیادہ ملنے وال...,https://jang.com.pk/news/1409376,امریکہ میں ہونے والے حالیہ انتخابات کے دوران ی...,world
3,603,بائیڈن کا عہدہ چھوڑنے سے قبل یوکرین کو اربوں ڈ...,https://jang.com.pk/news/1409373,وائٹ ہاؤس نے جنوری میں موجودہ صدر جو بائیڈن کے...,world
4,604,بابا صدیق کے قاتلوں سے کیا دینے کے وعدے کیے گئے؟,https://jang.com.pk/news/1409370,بابا صدیق کے قتل کے الزام میں ملوث ملزمان نے ت...,world


In [19]:
jang_df.shape[0]

411

In [10]:
jang_df.tail()

,id,title,link,content,gold_label
406,1006,انکم ٹیکس قواعد میں ترامیم، ایکٹو ٹیکس پیئرز ل...,https://jang.com.pk/news/1404129,فیڈرل بورڈ آف ریونیو (ایف بی آر) نے انکم ٹیک...,business
407,1007,بھارت نے پاکستان نہ آنے سے آئی سی سی کو آگاہ ک...,https://jang.com.pk/news/1409401,بھارتی کرکٹ بورڈ بی سی سی آئی نے انٹرنیشنل کرک...,business
408,1008,’طالع آزما قوتوں نے آئین پر عمل نہیں کیا، اسے ...,https://jang.com.pk/news/1409412,امیر جماعت اسلامی حافظ نعیم الرحمان نے کہا ہے ...,business
409,1009,کوئٹہ ریلوے اسٹیشن پر خودکش حملہ، 27 افراد جاں...,https://jang.com.pk/news/1409343,کوئٹہ ریلوے اسٹیشن پر ہونے والے خودکش حملے میں...,business
410,1010,سابق چیف جسٹس قاضی فائز عیسیٰ پر حملے سے میرا ...,https://jang.com.pk/news/1409402,لندن میں سابق چیف جسٹس قاضی فائز عیسیٰ پر ہونے...,business


In [21]:
def drop_last_n_rows(df, n=4):
    return df.iloc[:-n] if len(df) > n else pd.DataFrame()

jang_df = jang_df.groupby('gold_label', group_keys=False).apply(drop_last_n_rows, n=4)

# had to remove last 4 rows for each category of the jang df as they were getting repeated for some reason


TypeError: DataFrame.groupby() got an unexpected keyword argument 'include_groups'

In [22]:
jang_df.shape[0]

395

In [24]:
business_articles = jang_df[jang_df['gold_label'] == 'sports']
print(business_articles)

       id                                              title  \
104  1115  ورلڈ باڈی بلڈنگ چیمپئن شپ، پاکستان کے بلال احم...   
105  1116  کشمالہ طلعت 10 میٹر ایئر پسٹل میں ایشیا کی دوس...   
106  1117  بیس بال یونائیٹڈ عرب کلاسک: پاکستان نے سیمی فا...   
107  1118  پاکستان کے پاس آسٹریلیا میں 22 سال بعد تاریخ ...   
108  1119  پاکستانی گالفر احمد بیگ کی بین الاقوامی مقابلو...   
..    ...                                                ...   
198  1209  ہانگ کانگ سُپر سکسز: پاکستان نے بھارت کو آؤٹ ک...   
199  1210  سعودی عرب کی پاکستان سے ویمن فٹبال میچ کھیلنے ...   
200  1211  محمد حسنین کو آسٹریلیا کیخلاف پہلا ون ڈے کھلا...   
201  1212  محمد رضوان کو کپتان مقرر کرنا اچھا قدم ہے، شاد...   
202  1213  ورلڈ اسنوکر چیمپئن شپ، آصف نے فیصلہ کن راؤنڈ ...   

                                 link  \
104  https://jang.com.pk/news/1409425   
105  https://jang.com.pk/news/1409424   
106  https://jang.com.pk/news/1409419   
107  https://jang.com.pk/news/1409411   
108  https://jang.com.pk/n

In [27]:
express_df.shape[0]

350

In [28]:
dunya_df = scraper.get_dunya_articles()

Scraping articles from category 'Business'...
	--> Found 25 articles in 'Business'.
	--> Failed to scrape an article in 'Business': 'NoneType' object has no attribute 'find'
	--> Successfully scraped 24 articles from 'Business'.

Scraping articles from category 'Sports'...
	--> Found 25 articles in 'Sports'.
	--> Failed to scrape an article in 'Sports': 'NoneType' object has no attribute 'find'
	--> Successfully scraped 24 articles from 'Sports'.

Scraping articles from category 'Entertainment'...
	--> Found 25 articles in 'Entertainment'.
	--> Failed to scrape an article in 'Entertainment': 'NoneType' object has no attribute 'find'
	--> Successfully scraped 24 articles from 'Entertainment'.

Scraping articles from category 'World'...
	--> Found 25 articles in 'World'.
	--> Failed to scrape an article in 'World': 'NoneType' object has no attribute 'find'
	--> Successfully scraped 24 articles from 'World'.

Scraping articles from category 'Technology'...
	--> Found 25 articles in 'Techn

In [29]:
dunya_df.shape[0]

120

In [30]:
dunya_df.head()

,id,title,link,content,gold_label
0,1772,سٹاک مارکیٹ : رواں ہفتے 100 انڈیکس نے نئی بلند...,https://urdu.dunyanews.tv/index.php/ur/Busines...,کراچی :(ویب ڈیسک) پاکستان سٹاک ایکسچینج کے 100...,Business
1,1773,سونے کی فی تولہ قیمت 2 لاکھ 78 ہزار 800 روپے پ...,https://urdu.dunyanews.tv/index.php/ur/Busines...,کراچی: (ویب ڈیسک) عالمی و مقامی مارکیٹوں میں ہ...,Business
2,1774,گیس کے ذخائر میں مسلسل کمی، صارفین کو بجلی پر ...,https://urdu.dunyanews.tv/index.php/ur/Busines...,اسلام آباد: (ذیشان یوسفزئی) گیس کے مقامی ذخائر...,Business
3,1775,کے الیکٹرک ڈیفالٹرز کے بھی 68 ارب بروقت بل دین...,https://urdu.dunyanews.tv/index.php/ur/Busines...,اسلام آباد: (ذیشان یوسفزئی) کے الیکٹرک نے نادہ...,Business
4,1776,ایس آئی ایف سی کی مدد سے ملکی معیشت استحکام کی...,https://urdu.dunyanews.tv/index.php/ur/Busines...,اسلام آباد: (دنیا نیوز) خصوصی سرمایہ کاری سہو...,Business


In [35]:
business_articles = dunya_df[dunya_df['gold_label'] == 'Sports']
(business_articles)

,id,title,link,content,gold_label
24,1796,پاکستانی باکسرشعیب خان زہری نے انٹرنیشنل باکسن...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,مسقط: (دنیانیوز ) پاکستانی باکسر شعیب خان زہری...,Sports
25,1797,اے ٹی پی موزیل اوپن ٹینس ٹورنامنٹ: انگلینڈ کے ...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,پیرس: (ویب ڈیسک) انگلینڈ کے کیمرون نوری اے ٹی ...,Sports
26,1798,ورلڈ باڈی بلڈنگ اینڈ فزیق چیمپیئن شپ: پاکستانی...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,مالے :(ویب ڈیسک )جنوبی ایشیا کے ملک مالدیپ میں...,Sports
27,1799,سعودیہ میں ہونیوالا آئی ٹی ایف جونئیر ایونٹ پا...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,ریاض: (ویب ڈیسک) پاکستانی جوڑی نے سعودی عرب می...,Sports
28,1800,پاکستانی طیب اکرام بلا مقابلہ صدر انٹرنیشنل ہا...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,عمان :(ویب ڈیسک ) پاکستان کے طیب اکرام 8 برس ک...,Sports
29,1801,پاکستانی باکسرشعیب خان زہری نے انٹرنیشنل باکسن...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,مسقط: (دنیانیوز ) پاکستانی باکسر شعیب خان زہری...,Sports
30,1802,پاکستان آرمی نے مینز اینڈ ویمنز نیشنل سوئمنگ چ...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,لاہور: (دنیا نیوز) پاکستان آرمی نے مینز اینڈ و...,Sports
31,1803,مظفر گڑھ نویں تھل جیپ ریلی کے کوالیفائنگ راؤنڈ...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,مظفر گڑھ: (دنیا نیوز) مشکل ٹریک پر ڈرائیورز کا...,Sports
32,1804,بیس بال یونائیٹڈ عرب کلاسک ، پاکستان کی مسلسل ...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,دبئی : (ویب ڈیسک ) بیس بال یونائیٹڈ عرب کلاسک ...,Sports
33,1805,محمد آصف نے عالمی چیمپئن شپ جیت کرقوم کا سرفخر...,https://urdu.dunyanews.tv/index.php/ur/Sports/...,اسلام آباد: (دنیا نیوز) وزیراعظم محمد شہباز ش...,Sports


In [60]:
# EDA on dunya and geo content column(removed first 2 words as they contained cities of the news, not needed)
dunya_df['content'] = dunya_df['content'].str.replace(r'^([^\s]+\s+){2}', '', regex=True)
geo_df['content'] = geo_df['content'].str.replace(r'^([^\s]+\s+){2}', '', regex=True)


# print(dunya_df[['content']].head())

#combining the dataframe
combined_df = pd.concat([geo_df, jang_df, express_df, dunya_df], ignore_index=True)
# (combined_df.head())

#making all gold_labels lower case for consistency
combined_df['gold_label'] = combined_df['gold_label'].str.lower()
# combined_df.shape[0]

combined_df


,id,title,link,content,gold_label
0,0,ٹرمپ کے متنازع احکامات پر کیا ردعمل دینا ہے؟ پ...,https://urdu.geo.tv/latest/386198-,، 2024 امریکی وزارت دفاع پینٹاگون کے عہدیداران...,world
1,1,انسانی روبوٹ کا پہلا فن پارہ 30کروڑ روپے میں ن...,https://urdu.geo.tv/latest/386187-,، 2024 ایک انسان نما روبوٹ نے ایک انگریز ریاضی...,world
2,2,غزہ: مسلسل اسرائیلی بمباری سے گزشتہ 24 گھنٹوں ...,https://urdu.geo.tv/latest/386167-,، 2024 غزہ میں مسلسل اسرائیلی بمباری سے گزشتہ ...,world
3,3,امریکا نے قطر سے حماس قیادت کو ملک بدر کرنے کا...,https://urdu.geo.tv/latest/386165-,، 2024 امریکا نے اسرائیلی یرغمالیوں کی رہائی س...,world
4,4,امریکی مسلمان اور عرب ووٹرز کے ڈیموکریٹک پارٹی...,https://urdu.geo.tv/latest/386163-,، 2024 امریکا کی سب سے بڑی مسلم شہری حقوق اور ...,world
...,...,...,...,...,...
1460,1887,چیٹ جی پی ٹی سرچ انجن گوگل کو ٹکر دینے کیلئے تیار,https://urdu.dunyanews.tv/index.php/ur/Technol...,اور دیگر کمپنیاں اب تک اس کی مقبولیت کا مقابلہ...,technology
1461,1888,چین کی پہلی سویلین خاتون خلا میں پہنچ گئی,https://urdu.dunyanews.tv/index.php/ur/Technol...,میں کامیابی سے پہنچا دیا ہے۔ تینوں خلا بازوں ک...,technology
1462,1889,گوگل پر پوری دنیا کی دولت سے زائد کا جرمانہ عائد,https://urdu.dunyanews.tv/index.php/ur/Technol...,ڈالرز کا جرمانہ عائد کر دیا۔ رپورٹ کے مطابق رو...,technology
1463,1890,چار ماہ بعد ملک بھر میں انٹرنیٹ سروسز مکمل طور...,https://urdu.dunyanews.tv/index.php/ur/Technol...,کر دیا گیا۔ پاکستان ٹیلی کمیونیکیشن اتھارٹی (پ...,technology


In [66]:
geo_df['content'] = geo_df['content'].str.replace(r'^([^\s]+\s+){1}', '', regex=True)

In [67]:
#removing punctuation and numbers
import regex as re
def clean_text(x):
    return re.sub(r'[^\u0600-\u06FF\s]', '', x)

combined_df['content'] = combined_df['content'].apply(clean_text)

print(combined_df[['title', 'content']].head())

                                               title  \
0  ٹرمپ کے متنازع احکامات پر کیا ردعمل دینا ہے؟ پ...   
1  انسانی روبوٹ کا پہلا فن پارہ 30کروڑ روپے میں ن...   
2  غزہ: مسلسل اسرائیلی بمباری سے گزشتہ 24 گھنٹوں ...   
3  امریکا نے قطر سے حماس قیادت کو ملک بدر کرنے کا...   
4  امریکی مسلمان اور عرب ووٹرز کے ڈیموکریٹک پارٹی...   

                                             content  
0  ،  امریکی وزارت دفاع پینٹاگون کے عہدیداران میں...  
1  ،  ایک انسان نما روبوٹ نے ایک انگریز ریاضی دان...  
2  ،  غزہ میں مسلسل اسرائیلی بمباری سے گزشتہ  گھن...  
3  ،  امریکا نے اسرائیلی یرغمالیوں کی رہائی سے مت...  
4  ،  امریکا کی سب سے بڑی مسلم شہری حقوق اور وکال...  


In [68]:
combined_df['gold_label'] = combined_df['gold_label'].replace('technology', 'science-technology')

In [69]:
combined_df.tail

<bound method NDFrame.tail of         id                                              title  \
0        0  ٹرمپ کے متنازع احکامات پر کیا ردعمل دینا ہے؟ پ...   
1        1  انسانی روبوٹ کا پہلا فن پارہ 30کروڑ روپے میں ن...   
2        2  غزہ: مسلسل اسرائیلی بمباری سے گزشتہ 24 گھنٹوں ...   
3        3  امریکا نے قطر سے حماس قیادت کو ملک بدر کرنے کا...   
4        4  امریکی مسلمان اور عرب ووٹرز کے ڈیموکریٹک پارٹی...   
...    ...                                                ...   
1460  1460  چیٹ جی پی ٹی سرچ انجن گوگل کو ٹکر دینے کیلئے تیار   
1461  1461          چین کی پہلی سویلین خاتون خلا میں پہنچ گئی   
1462  1462   گوگل پر پوری دنیا کی دولت سے زائد کا جرمانہ عائد   
1463  1463  چار ماہ بعد ملک بھر میں انٹرنیٹ سروسز مکمل طور...   
1464  1464  ٹیکنالوجی کی وجہ سے ہر شعبےمیں تبدیلیاں ہو رہی...   

                                                   link  \
0                    https://urdu.geo.tv/latest/386198-   
1                    https://urdu.geo.tv/latest/386187-   
2           

In [70]:
combined_df = combined_df.drop(columns=['id'])

combined_df['id'] = range(len(combined_df))
cols = ['id'] + [col for col in combined_df.columns if col != 'id']

combined_df = combined_df[cols]

combined_df

# adding id for each article

,id,title,link,content,gold_label
0,0,ٹرمپ کے متنازع احکامات پر کیا ردعمل دینا ہے؟ پ...,https://urdu.geo.tv/latest/386198-,، امریکی وزارت دفاع پینٹاگون کے عہدیداران میں...,world
1,1,انسانی روبوٹ کا پہلا فن پارہ 30کروڑ روپے میں ن...,https://urdu.geo.tv/latest/386187-,، ایک انسان نما روبوٹ نے ایک انگریز ریاضی دان...,world
2,2,غزہ: مسلسل اسرائیلی بمباری سے گزشتہ 24 گھنٹوں ...,https://urdu.geo.tv/latest/386167-,، غزہ میں مسلسل اسرائیلی بمباری سے گزشتہ گھن...,world
3,3,امریکا نے قطر سے حماس قیادت کو ملک بدر کرنے کا...,https://urdu.geo.tv/latest/386165-,، امریکا نے اسرائیلی یرغمالیوں کی رہائی سے مت...,world
4,4,امریکی مسلمان اور عرب ووٹرز کے ڈیموکریٹک پارٹی...,https://urdu.geo.tv/latest/386163-,، امریکا کی سب سے بڑی مسلم شہری حقوق اور وکال...,world
...,...,...,...,...,...
1460,1460,چیٹ جی پی ٹی سرچ انجن گوگل کو ٹکر دینے کیلئے تیار,https://urdu.dunyanews.tv/index.php/ur/Technol...,اور دیگر کمپنیاں اب تک اس کی مقبولیت کا مقابلہ...,science-technology
1461,1461,چین کی پہلی سویلین خاتون خلا میں پہنچ گئی,https://urdu.dunyanews.tv/index.php/ur/Technol...,میں کامیابی سے پہنچا دیا ہے۔ تینوں خلا بازوں ک...,science-technology
1462,1462,گوگل پر پوری دنیا کی دولت سے زائد کا جرمانہ عائد,https://urdu.dunyanews.tv/index.php/ur/Technol...,ڈالرز کا جرمانہ عائد کر دیا۔ رپورٹ کے مطابق رو...,science-technology
1463,1463,چار ماہ بعد ملک بھر میں انٹرنیٹ سروسز مکمل طور...,https://urdu.dunyanews.tv/index.php/ur/Technol...,کر دیا گیا۔ پاکستان ٹیلی کمیونیکیشن اتھارٹی پی...,science-technology


In [74]:
combined_df['content'] = combined_df['content'].str.replace('،', '', regex=False)
#removing commas from here

# Output
- Save a combined csv of all 3 sites.

In [75]:
combined_df.to_csv('output_file.csv', index=False, encoding='utf-8')